# Bell Manual

## 环境配置
目前使用的包很少，python 3.10+，根据提示安装相应的包即可。

## 0. 文件树
所有文件的说明参考`BQIC/README.md`，此处使用的文件根目录是CompilerDev。

## 1. 序列定义
参考`Config/DDS_config_example.py`,设置
1. 时序单元：通道名对应的频率、幅度和相位等状态参数
2. 时间序列：根据需要将时序单元拼接成时间序列
3. 可以在其中设置参数，就像函数的参数一样，方便后续调用

In [ ]:
Thresh = {"493":0.14,"553":0.18,'614':0.35,'650':0.3,'1762EOM':0.2,'532':0.2}
# Pumping States
Pumping = DDS("Pumping")
Pumping.s0.f(100).a(0.2)
Pumping.s1.f(220).a(0.75)
Pumping.s2.f(110).a(0.5)
Pumping.s3.f(110).a(0.25)
Pumping.s4.f(220).a(1)
Pumping.s5.f(110).a(0.75)
Pumping.s6.f(220).a(0.5)
Pumping.s7.f(220).a(0.25)
Pumping.s8.f(220).a(0)
Pumping.on.f(120).a(0.1)
Pumping.on.f(125).a(0.5*Thresh['493'])
Pumping.off.f(125).a(0)

## 2.生成硬件指令和数据
1. 使用`Interpreter/Compile2Bell_data.py`生成硬件需要的数据
2. 使用`Interpreter/Compile2Bell_Inst.py`生成硬件需要的指令
3. 生成的数据和指令保存在`Output`文件夹中的`Datawithheader.txt`和`Inst.txt`中

In [1]:
# Compile2Bell_data.py
#0. 导入必要的模块
import sys
import os
# 在 Jupyter Notebook 中获取当前工作目录
current_dir = os.getcwd()
print("Current working directory:", current_dir)

# 根据实际情况调整项目根目录
if 'Interpreter' in current_dir:
    project_root = os.path.abspath(os.path.join(current_dir, '..'))
else:
    project_root = os.path.abspath(os.path.join(current_dir, 'CompilerDev'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("project_root:", project_root)
print("sys.path:", sys.path[:3])  # 只显示前3个路径

# 确保 Opfunc 目录下有 __init__.py 文件
opfunc_init = os.path.join(project_root, 'Opfunc', '__init__.py')
if not os.path.exists(opfunc_init):
    print(f"Warning: {opfunc_init} does not exist. Creating empty __init__.py")
    os.makedirs(os.path.dirname(opfunc_init), exist_ok=True)
    with open(opfunc_init, 'w') as f:
        pass
    
from DDS_Seq import DDS,Seq
from config418 import * 

import CompileUtils as utils

from Opfunc.OpfuncPulse import OpfuncPulse
from Opfunc.OpfuncRF_DDS import OpfuncRF_DDS

from typing import List

Current working directory: d:\code\BQIC\CompilerDev\Interpreter
project_root: d:\code\BQIC\CompilerDev
sys.path: ['d:\\code\\BQIC\\CompilerDev', 'd:\\code\\BQIC\\CompilerDev\\Interpreter', 'd:\\tools\\Scoop\\apps\\miniconda3\\23.5.2-0\\python310.zip']


In [3]:
#1.init seq 导入一个Config文件中定义好的时间序列或者基于已有的时序单元设计一个新的时间序列
seq_cooling = Seq().Operate(4000).Detection(10000,10).Operate_0(5000)

#2. 对序列进行编译，转换成需要的中间表示
seqs4Bell = seq_cooling.seq
seqsIR = utils.compile2Bell(seqs4Bell)  # Bell IR

#3. 定义Bell设备的DDS和TTL通道
dds: List[OpfuncRF_DDS] = [OpfuncRF_DDS(DeviceID=i) for i in range(4)] # Bell设备有4个DDS通道，每个通道上有6个频率，TODO：扩充通道数
ttl: List[OpfuncPulse] = [OpfuncPulse(DeviceID=i) for i in range(32)]

#4. 需要将IR指令转换为设备data数据
OutputDir = "../Output/"
OutputPath = OutputDir + "Data0.txt"
OutputPathwithHeader = OutputDir + "DatawithHeader0.txt"

with open(OutputPath, "w") as f: # 打开dds.txt文件，并将其清空
    pass

for seq in seqsIR:
    for i in range(len(dds)):
        dds[i].reset()
        flag_write = False
        for item in seq:
            if item[0]=="dds" and item[1]==i:
                dds[i].setwaveform(item[2], item[3], item[4], item[5]) # TODO: 只有一个频率分量
                dds[i].gen_assembler()
                dds[i].setwaveform(0,0,0,0)
                dds[i].gen_assembler()
                dds[i].gen_assembler()
                dds[i].gen_assembler()
                dds[i].gen_assembler()
                dds[i].gen_assembler()
                dds[i].adjust_array_length() #add EOF
                flag_write = True
                break # 找到对应的dds通道后，跳出循环
        # 如果没有找到对应的dds通道，则将其设置为0
        if flag_write==False:
            dds[i].setwaveform(0,0,0,0)
            dds[i].gen_assembler()
            dds[i].gen_assembler()
            dds[i].gen_assembler()
            dds[i].gen_assembler()
            dds[i].gen_assembler()
            dds[i].gen_assembler()
            dds[i].adjust_array_length() #add EOF

    #5. 输出到文件    
    for i in range(len(dds)):
        dds[i].output2file(OutputPath) # 输出到文件

    for i in range(len(ttl)):
        ttl[i].reset()
        for item in seq:
            if item[0]=="TTL" and item[1]==i:
                # print("item:", item)
                ttl[i].setwaveform("input", 0,1, item[2])  # 设置为输出模式
                ttl[i].gen_assembler()
                ttl[i].adjust_array_length() #add EOF
# 6. 合并文件头
utils.merge_param_files_with_header(OutputPath, OutputPathwithHeader)

添加header成功，文件已保存到: ../Output/DatawithHeader0.txt
header: eb9c55aa000200000000000000700000


In [4]:
# Compile2Bell_Inst.py
# 0. 导入需要用到指令模块
import Compile2Bell_Inst

# 1. 定义IR指令长度,可以从Compile2Bell_Data.py中获取
IR_length = len(seqsIR)
print("IR_length:", IR_length)
# 2. 定义IR指令
Inst = Compile2Bell_Inst.LoadDDS(IR_length)
# 3. 写入文件
OutputDir = "../Output/"
OutputPath = OutputDir + "Inst0.txt"
with open(OutputPath, "w") as f:
    for item in Inst[0]:
        f.write(item + "\n")

IR_length: 4


## 3.PXIE接口上传运行

In [ ]:
# 参考PXIEInterfaceAPI模块
